### Install required packages

In [ ]:
!pip install replicate
#Documentation -https://replicate.com/meta/llama-2-70b/api?tab=python

### Import required libraries

In [3]:
import os
import replicate
import csv
import time
import pandas as pd
from datetime import datetime
from google.colab import userdata
import json

### Set the API KEY using Google colab user data option

In [4]:
replicate_access = userdata.get('replicate')
os.environ['REPLICATE_API_KEY'] = replicate_access

### Read the configurations

In [8]:
json_file_path = "/content/drive/MyDrive/Colab Notebooks/config.json"
with open(json_file_path, "r") as file:
    configuration = json.load(file)

### Load the user story descriptions to data frame

In [9]:
input_csv_path = configuration['input_csv_path']
# Read the complete CSV file as a pandas DataFrame
data_frame = pd.read_csv(input_csv_path)

## STEPS
### 1.   Read each user story description
### 2.   Set the model [llama2-13B] and model parameters
### 3.   Read the zero, few-shot prompt from configuration
### 4.   Call replica chat completion end point
### 5. Save the response as .feature file

In [ ]:
# Specify the columns to iterate
#step 1
us_desc_column=configuration['us_desc_column']
app_desc_column=configuration['app_desc_column']
# read model, configuration parameters
#step 2
llama2_model=configuration['llama2_model']
llama2_temperature=configuration['llama2_temperature']
llama2_top_p=configuration['llama2_top_p']
llama2_max_tokens=configuration['llama2_max_tokens']
response_feature_file_llama2=configuration['response_feature_file_llama2']
few_shot_method=configuration['few_shot_method']
zero_shot_method=configuration['zero_shot_method']
output_csv_path=configuration['output_csv_path']
#step 3
system_prompt=configuration['system_prompt']
# Iterate over each row to read user story descriptions
for index, row in data_frame.iterrows():
    user_story = row[us_desc_column]
    story_context = row[app_desc_column]
    #step 3
    zero_shot_array=configuration['zero_shot']
    zero_shot=' '.join(zero_shot_array)
    few_shot_user_prompt_array=configuration['few_shot_user_prompt']
    few_shot_user_prompt=' '.join(few_shot_user_prompt_array)
    few_shot_assistant_prompt_array=configuration['few_shot_assistant_prompt']
    few_shot_assistant_prompt=' '.join(few_shot_assistant_prompt_array)
    user_message = zero_shot+user_story
    api = replicate.Client(api_token=os.environ['REPLICATE_API_KEY'])
    prompt = f"{system_prompt}\n{few_shot_user_prompt}\n{few_shot_assistant_prompt}\n{user_message}"
    # step 4
    output = api.run(
        "meta/llama-2-13b-chat",
       #"meta/llama-2-7b-chat",
       # input={"prompt": user_message,
         input={
                 "prompt":prompt,
                 "temperature": llama2_temperature,
                 "max_new_tokens": llama2_max_tokens,
                 "top_p":llama2_top_p,
               }
        )
    output_text=''
    output_text = ''.join(output)
    time.sleep(20)
    # Specify the output file name
    #step 5s
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_name = f"US{index}_{timestamp}_{few_shot_method}_{response_feature_file_llama2}.feature"
        # Specify the path where you want to save the output file
    output_file_path = os.path.join(output_csv_path, output_file_name)
    # Write the output texts to a text file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(output_text)